In [38]:
import pandas as pd
import numpy as np
import logging
import sys
import os
import scipy
from sklearn import linear_model
pd.options.display.max_rows = 999
pd.set_option('display.max_columns',500)

import bokeh
from plotly.offline import plot, iplot, init_notebook_mode
from plotly.offline import download_plotlyjs 
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("regress")

In [4]:
df_train = pd.read_csv("train.csv")

In [5]:
df_train.columns = df_train.columns.str.lower()

In [6]:
outside_features = ['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'lotshape', 'landcontour', 'bldgtype',
 'landslope', 'housestyle', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd', 'roofstyle', 'roofmatl',
 'exterior1st', 'masvnrtype', 'masvnrarea', 'exterqual', 'extercond', 'foundation', 'mosold',
 'yrsold', 'saletype', 'salecondition', 'saleprice']

In [7]:
out_df = df_train[outside_features].copy()

In [8]:
lotshape_df = pd.DataFrame(columns=["lotshape","lotshape_r","lotshape_desc"],
                           data=[["Reg",0,"Regular"],["IR1",1, "Slightly Irregular"],
                                 ["IR2",2,"Moderately Irregular"],["IR3",3, "Irregular"]])

In [9]:
landcontour_df = pd.DataFrame(columns=["landcontour","landcontour_r","landcontour_desc"],
                           data=[["Lvl",0,"Near Flat/Level"],["Bnk",1, "anked - Quick and significant rise from street grade to building"],
                                 ["Low",2,"Depression"],["HLS",3, "Hillside - Significant slope from side to side"]])

In [10]:
bldgtype_df = pd.DataFrame(columns=["bldgtype","bldgtype_r","bldgtype_desc"],
                           data=[["1Fam",0,"Single-family Detached"],["2fmCon",1, "Two-family Conversion"],
                                 ["Duplex",2,"Duplex"],["TwnhsE",3, "Townhouse End Unit"]
                                , ["Twnhs",4, "ownhouse Inside Unit"]])

In [11]:
landslope_df = pd.DataFrame(columns=["landslope","landslope_r","landslope_desc"],
                           data=[["Gtl",0,"Gentle slope"],["Mod",1, "Moderate Slope"],
                                 ["Sev",2,"Severe Slope"]])

In [12]:
housestyle_df = pd.DataFrame(columns=["housestyle","housestyle_r","housestyle_desc"],
                           data=[ ["1Story",0,"One story"]
                                 ,["1.5Fin",1, "One and one-half story: 2nd level finished"]
                                 ,["1.5Unf",2,"One and one-half story: 2nd level unfinished"]
                                 ,["2Story",3, "Two Storey"]
                                 ,["2.5Fin",5, "Two and one-half story: 2nd level finished"]
                                 ,["2.5Unf",6, "wo and one-half story: 2nd level unfinished"]
                                 ,["SFoyer",7, "Split Foyer"]
                                 ,["SLvl",8, "Split Level"]                                
                                ])
# Justification: You have more control over the recoding. 
# For example, 2storey and 2.5 storey can be recoded as a single variable

In [13]:
roofstyle_df = pd.DataFrame(columns=["roofstyle","roofstyle_r","roofstyle_desc"],
                           data=[ ["Flat",0,"Flat"]
                                 ,["Gable",1, "Gable"]
                                 ,["Gambrel",2,"Gambrel"]
                                 ,["Hip",3,"Hip"]
                                 ,["Mansard",4,"Mansard"]
                                 ,["Shed",5,"Shed"]                                 
                                ])

In [14]:
roofmatl_df = pd.DataFrame(columns=["roofmatl","roofmatl_r","roofmatl_desc"],
                           data=[ ["ClyTile",0,"Clay or Tile"]
                                 ,["CompShg",1, "Standard (Composite) Shingle"]
                                 ,["Membran",2,"Membran"]
                                 ,["Metal",3,"Metal"]
                                 ,["Roll",4,"Roll"]
                                 ,["Tar&Grv",5,"Gravel & Tar"]
                                 ,["WdShake",6,"Wood Shakes"]                                 
                                 ,["WdShngl",7,"Wood Shingles"]                                 
                                ])

In [15]:
exterior1st_df = pd.DataFrame(columns=["exterior1st","exterior1st_r","exterior1st_desc"],
                           data=[ ["AsbShng",0,"Asbestos Shingles"]
                                 ,["AsphShn",1, "Asphalt Shingles"]
                                 ,["BrkComm",2,"Brick Common"]
                                 ,["BrkFace",3,"Brick Face"]
                                 ,["CBlock",4,"Cinder Block"]
                                 ,["CemntBd",5,"Cement Board"]                                 
                                 ,["HdBoard",5,"Hard Board"]                                 
                                 ,["ImStucc",5,"Imitation Stucco"]                                 
                                 ,["MetalSd",5,"Metal Siding"]                                 
                                 ,["Other",5,"Other"]                                 
                                 ,["Plywood",5,"Plywood"]                                 
                                 ,["PreCast",5,"PreCast"]                                 
                                 ,["Stone",5,"Stone"]                                 
                                 ,["Stucco",5,"Stucco"]                                 
                                 ,["VinylSd",5,"Vinyl Siding"]                                 
                                 ,["Wd Sdng",5,"Wood Siding"]                                 
                                 ,["WdShing",5,"Wood Shingles"]                                 
                                ])

In [16]:
masvnrtype_df = pd.DataFrame(columns=["masvnrtype","masvnrtype_r","masvnrtype_desc"],
                           data=[ ["BrkCmn",0,"Brick Common"]
                                 ,["BrkFace",1, "Brick Face"]
                                 ,["CBlock",2,"Cinder Block"]
                                 ,["None",3,"None"]
                                 ,["Stone",4,"Stone"]                                 
                                ])

In [17]:
exterqual_df = pd.DataFrame(columns=["exterqual","exterqual_r","exterqual_desc"],
                           data=[ ["Ex",2,"Excellent"]
                                 ,["Gd",1, "Good"]
                                 ,["TA",0,"Average"]
                                 ,["Fair",-1,"Fair"]
                                 ,["Po",-2,"Poor"]                                 
                                ])

In [18]:
extercond_df = pd.DataFrame(columns=["extercond","extercond_r","extercond_desc"],
                           data=[ ["Ex",2,"Excellent"]
                                 ,["Gd",1, "Good"]
                                 ,["TA",0,"Average"]
                                 ,["Fair",-1,"Fair"]
                                 ,["Po",-2,"Poor"]                                 
                                ])

In [19]:
foundation_df = pd.DataFrame(columns=["foundation","foundation_r","foundation_desc"],
                           data=[ ["BrkTil",0,"Brick & Tile"]
                                 ,["CBlock",1, "Cinder Block"]
                                 ,["PConc",2,"Poured Contrete"]
                                 ,["Slab",3,"Slab"]
                                 ,["Stone",4,"Stone"]                                 
                                 ,["Wood",5,"Wood"]                                 
                                ])

In [20]:
saletype_df = pd.DataFrame(columns=["saletype","saletype_r","saletype_desc"],
                           data=[ ["WD",0,"Warranty Deed - Conventional"]
                                 ,["CWD",1, "Warranty Deed - Cash"]
                                 ,["VWD",2,"Warranty Deed - VA Loan"]
                                 ,["New",3,"Home just constructed and sold"]
                                 ,["COD",4,"Court Officer Deed/Estate"]                                 
                                 ,["Con",5,"Contract 15% Down payment regular terms"]                              
                                 ,["ConLw",6,"Contract Low Down payment and low interest"]                              
                                 ,["ConLI",7,"Contract Low Interest"]                              
                                 ,["ConLD",8,"Contract Low Down"]                              
                                 ,["Oth",9,"Other"]                                        
                                ])

In [21]:
salecondition_df = pd.DataFrame(columns=["salecondition","salecondition_r","salecondition_desc"],
                           data=[ ["Normal",0,"Normal Sale"]
                                 ,["Abnorml",1, "Abnormal Sale -  trade, foreclosure, short sale"]
                                 ,["AdjLand",2,"Adjoining Land Purchase"]
                                 ,["Alloca",3,"Allocation - two linked properties with separate deeds, typically condo with a garage unit"]
                                 ,["Family",4,"Sale between family members"]                                 
                                 ,["Partial",5,"Home was not completed when last assessed (associated with New Homes)"]                                                                  
                                ])

In [22]:
out_df = pd.merge(out_df,lotshape_df,on=['lotshape'], how='left' )
out_df = pd.merge(out_df,landcontour_df,on=['landcontour'], how='left' )
out_df = pd.merge(out_df,bldgtype_df,on=['bldgtype'], how='left' )
out_df = pd.merge(out_df,landslope_df,on=['landslope'], how='left' )
out_df = pd.merge(out_df,housestyle_df,on=['housestyle'], how='left' )
out_df = pd.merge(out_df,roofstyle_df,on=['roofstyle'], how='left' )
out_df = pd.merge(out_df,roofmatl_df,on=['roofmatl'], how='left' )
out_df = pd.merge(out_df,exterior1st_df,on=['exterior1st'], how='left' )
out_df = pd.merge(out_df,masvnrtype_df,on=['masvnrtype'], how='left' )
out_df = pd.merge(out_df,exterqual_df,on=['exterqual'], how='left' )
out_df = pd.merge(out_df,extercond_df,on=['extercond'], how='left' )
out_df = pd.merge(out_df,foundation_df,on=['foundation'], how='left' )
out_df = pd.merge(out_df,saletype_df,on=['saletype'], how='left' )
out_df = pd.merge(out_df,salecondition_df,on=['salecondition'], how='left' )

In [23]:
out_df['day'] = 1
out_df['month'] = out_df.mosold
out_df['year'] = out_df.yrsold
out_df['monthyear'] = pd.to_datetime(out_df[['year','month','day']], format='%Y%m')

In [24]:
out_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,lotshape,landcontour,bldgtype,landslope,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,masvnrtype,masvnrarea,exterqual,extercond,foundation,mosold,yrsold,saletype,salecondition,saleprice,lotshape_r,lotshape_desc,landcontour_r,landcontour_desc,bldgtype_r,bldgtype_desc,landslope_r,landslope_desc,housestyle_r,housestyle_desc,roofstyle_r,roofstyle_desc,roofmatl_r,roofmatl_desc,exterior1st_r,exterior1st_desc,masvnrtype_r,masvnrtype_desc,exterqual_r,exterqual_desc,extercond_r,extercond_desc,foundation_r,foundation_desc,saletype_r,saletype_desc,salecondition_r,salecondition_desc,day,month,year,monthyear
0,1,60,RL,65.0,8450,Reg,Lvl,1Fam,Gtl,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,BrkFace,196.0,Gd,TA,PConc,2,2008,WD,Normal,208500,0,Regular,0,Near Flat/Level,0,Single-family Detached,0,Gentle slope,3,Two Storey,1,Gable,1,Standard (Composite) Shingle,5,Vinyl Siding,1.0,Brick Face,1.0,Good,0.0,Average,2,Poured Contrete,0,Warranty Deed - Conventional,0,Normal Sale,1,2,2008,2008-02-01
1,2,20,RL,80.0,9600,Reg,Lvl,1Fam,Gtl,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,None,0.0,TA,TA,CBlock,5,2007,WD,Normal,181500,0,Regular,0,Near Flat/Level,0,Single-family Detached,0,Gentle slope,0,One story,1,Gable,1,Standard (Composite) Shingle,5,Metal Siding,3.0,None,0.0,Average,0.0,Average,1,Cinder Block,0,Warranty Deed - Conventional,0,Normal Sale,1,5,2007,2007-05-01
2,3,60,RL,68.0,11250,IR1,Lvl,1Fam,Gtl,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,BrkFace,162.0,Gd,TA,PConc,9,2008,WD,Normal,223500,1,Slightly Irregular,0,Near Flat/Level,0,Single-family Detached,0,Gentle slope,3,Two Storey,1,Gable,1,Standard (Composite) Shingle,5,Vinyl Siding,1.0,Brick Face,1.0,Good,0.0,Average,2,Poured Contrete,0,Warranty Deed - Conventional,0,Normal Sale,1,9,2008,2008-09-01
3,4,70,RL,60.0,9550,IR1,Lvl,1Fam,Gtl,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,None,0.0,TA,TA,BrkTil,2,2006,WD,Abnorml,140000,1,Slightly Irregular,0,Near Flat/Level,0,Single-family Detached,0,Gentle slope,3,Two Storey,1,Gable,1,Standard (Composite) Shingle,5,Wood Siding,3.0,None,0.0,Average,0.0,Average,0,Brick & Tile,0,Warranty Deed - Conventional,1,"Abnormal Sale - trade, foreclosure, short sale",1,2,2006,2006-02-01
4,5,60,RL,84.0,14260,IR1,Lvl,1Fam,Gtl,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,BrkFace,350.0,Gd,TA,PConc,12,2008,WD,Normal,250000,1,Slightly Irregular,0,Near Flat/Level,0,Single-family Detached,0,Gentle slope,3,Two Storey,1,Gable,1,Standard (Composite) Shingle,5,Vinyl Siding,1.0,Brick Face,1.0,Good,0.0,Average,2,Poured Contrete,0,Warranty Deed - Conventional,0,Normal Sale,1,12,2008,2008-12-01


In [25]:
out_df.drop(['day','month','year'], axis=1, inplace=True)

In [26]:
out_df[out_df.landslope.isnull()].shape

(0, 56)

In [27]:
lnr = linear_model.LinearRegression()
lnr.fit(out_df[['lotarea', 'lotshape_r','landcontour_r']], out_df[['saleprice']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
lnr.coef_

array([[  1.58682015e+00,   2.79712444e+04,  -9.96539038e+01]])

In [29]:
lnr.intercept_

array([ 152833.93576631])

In [30]:
lnr.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'normalize': False}

In [31]:
lnr.score(out_df[['lotarea', 'lotshape_r','landcontour_r']], out_df[['saleprice']])

0.1074252607570737